# Debugging

Objectives

今天课程的主题是调试。

首先，我们将探讨如何避免调试--编写代码的方法要么能完全避免调试，要么至少能在必须调试时让调试变得简单。

但有时你别无选择，只能进行调试--尤其是在将整个系统连接在一起时才发现的错误，或者是在系统部署后由用户报告的错误，在这种情况下，可能很难将错误定位到某个特定的模块。在这种情况下，我们可以建议采用系统化的策略来提高调试效率。

Andreas Zeller 所著的[Why Programs Fail ](https://mit.primo.exlibrisgroup.com/permalink/01MIT_INST/sjd9fk/cdi_safari_books_v2_9780123745156)是一本关于系统调试的好书。这本书中的很多内容都是受这本书的启发。

John Regehr 的 ["How to Debug"](https://blog.regehr.org/archives/199)也与此相关，它是嵌入式系统课程的讲义，比 6.102 更低级，但具有相同的系统调试一般原则。

最后，[ Debugging: The Nine Indispensable Rules for Finding Even the Most Elusive Software and Hardware Problems](https://mit.primo.exlibrisgroup.com/permalink/01MIT_INST/sjd9fk/cdi_skillsoft_books24x7_bks00002516)是一本可读性强、非常实用的指南，适用于从软件到硬件、从汽车到管道等各种技术环境下的调试。我们在本篇阅读中参考了 ["Nine Rules"](http://www.debuggingrules.com/Debugging_CH2.PDF#page=2)中的几条。

## First defense: make bugs impossible

抵御 bug 的最好办法就是在设计上让它们不可能出现。

我们已经讨论过的一种方法就是静态检查。静态检查通过在编译时捕获错误来消除许多错误。
| “Making mistakes is hard to avoid but not caring to prevent mistakes is unacceptable.”|
|---|
|Zeller, Why Programs Fail, Chapter 3|

我们在前面的阅读中也看到了一些动态检查的例子。例如，Python 通过动态捕获使数组溢出 bug 变得不可能。如果您试图使用超出 list 边界的索引，Python 会自动产生错误。JavaScript 则没有这么好，因为它会在读取错误时悄悄返回未定义。C 和 C++ 等老式语言的情况更糟，它们会默许超出固定大小数组边界的错误写入，从而导致错误和安全漏洞。

Immutability 是另一个防止漏洞的设计原则，它有两种类型：immutable types and unreassignable references（常量）。

字符串是一种不可变类型。在字符串上调用任何方法都不会改变字符串所代表的字符序列。字符串可以被传递和共享，而不用担心会被其他代码修改。

TypeScript 还提供了不可赋值引用：使用关键字 const 声明的变量，可以赋值一次，但永远不能再赋值。在声明局部变量时，尽可能使用 const 是一种很好的做法。与变量类型一样，这些声明也是重要的文档，对代码读者有用，编译器也会对其进行静态检查。

请看下面这个例子

```ts
const letters: Array<string> = ['a', 'e', 'i', 'o', 'u'];
```
letters变量被声明为 const，但它真的不变吗？下面哪些语句是非法的（被编译器静态捕获），哪些是允许的？

```ts
letters = ['x', 'y', 'z']; 
letters[0] = 'z';
```
您可以在下面的练习中找到答案。请注意 const 的含义！它只是使引用不可赋值，但引用指向的对象可能是可变的。

## Second defense: make bugs easy to find
静态检查、动态检查和immutability只能在一定程度上预防程序错误的出现，因此我们的下一个设计策略就是在程序错误不可避免地出现时，让它们更容易被发现。我们可以将错误定位到程序的一小部分，这样我们就不必在太广的范围内寻找错误的原因。如果将错误定位在单个方法或小模块上，只需研究程序文本就能发现错误。

我们已经讨论过 "fail fast"：越早发现问题（越接近其原因），就越容易修复。

让我们从一个简单的例子开始：
```ts
/**
 * @param x  requires x >= 0
 * @returns approximation to square root of x
 */
function sqrt(x: number): number { ... }

```

现在假设有人用一个负参数调用 sqrt。sqrt 的好的行为是什么？由于调用者没有满足 x 应该是非负值的要求，因此 sqrt 不再受其合同条款的约束，所以从技术上讲，它可以为所欲为：返回任意值，或进入无限循环，或关闭 CPU。然而，由于错误调用表明调用者存在错误，因此最有用的行为是尽早指出错误。为此，我们可以在运行时插入对前提条件的检查。下面是一种检查方法：

```ts
/**
 * @param x  requires x >= 0
 * @returns approximation to square root of x
 */
function sqrt(x: number): number { 
    if ( ! (x >= 0)) throw new Error("required x >= 0, but was: " + x);
    ...
}
```

当前提条件不满足时，该代码会抛出错误终止程序。调用者错误的影响不会传播。

检查前提条件是防御性编程的一个例子。真实的程序很少没有错误。Defensive programming提供了一种方法，即使你不知道错误在哪里，也能减轻错误的影响。

### Assertions
通常的做法是为这类防御检查定义一个函数，通常称为 assert。这种方法抽象出了断言失败时到底会发生什么。失败的断言可能会退出；可能会在日志文件中记录事件；可能会通过电子邮件向维护者发送报告。

JavaScript/TypeScript 没有内置断言，但我们将在 6.102 中使用 Node 的断言包。断言的最简单形式是使用布尔表达式，如果布尔表达式求值为 false，则抛出 AssertionError：
```ts
assert(x >= 0);
```
断言的另一个好处是，它记录了程序在这一点上的状态假设。对于阅读你代码的人来说，assert(x >= 0) 表示 "在这一点上，x >= 0 应该始终为真"。但与注释不同的是，断言是在运行时执行假设的可执行代码。

断言还可以包含字符串信息，在断言失败时打印出来。这可以用来向程序员提供有关失败原因的更多细节。例如

```ts
assert(x >= 0, "x is " + x);
```
If x === -1, then this assertion fails with the error message

> x is -1

配合堆栈跟踪，堆栈跟踪会告诉你断言是在代码中的哪个位置发现的，以及将程序带到这个位置的调用顺序。这些信息通常足以帮助您开始查找错误。

关于断言需要注意的一点是，在许多语言（如 Java）中，断言默认是关闭的。这意味着，如果你像往常一样运行程序，你的断言都不会被检查！语言设计者选择这样做的一个原因是，检查断言有时会对性能造成损失。例如，一个使用二进制搜索数组的函数要求对数组进行排序。然而，断言这一要求需要扫描整个数组，这就将本应在对数时间内运行的操作变成了需要线性时间的操作。在测试过程中，你应该愿意（渴望！）付出这样的代价，因为它能让调试变得更容易，但在程序发布给用户后就不一样了。不过，对于大多数应用程序来说，断言与代码的其他部分相比并不昂贵，而且它们在错误检查方面带来的好处也值得在性能上付出这么小的代价。

在默认关闭断言的语言中，请确保显式启用断言。我们在 6.102 中使用的 Node 断言库始终运行其断言语句，因此无需担心显式开启断言。

### What to assert

以下是您应该断言的一些事项：

**方法参数要求**，如上面的 sqrt。

**方法返回值要求**。这种断言有时称为自我检查。例如，sqrt 方法可能会对其结果进行平方，以检查它是否合理地接近 x：

```ts
function sqrt(x: number): number {
    assert(x >= 0);
    let r: number;
    ... // compute result r
    assert(Math.abs(r*r - x) < .0001);
    return r;
}
```
何时应编写运行时断言？在编写代码时，而不是事后。在编写代码时，您会考虑invariants。如果推迟编写断言，就不太可能做到这一点，而且很可能会遗漏一些重要的不变式。

### What not to assert
Runtime assertions不是免费的。它们会使代码变得杂乱无章，因此必须谨慎使用。避免使用琐碎的断言，就像避免使用无意义的注释一样。例如

```ts
// don't do this:
x = y + 1;
assert(x === y+1);
```

该断言不会发现代码中的错误。它只会发现编译器或 JavaScript 解释器中的错误，在没有充分理由怀疑编译器或 JavaScript 解释器之前，您应该信任它们。如果断言在本地上下文中是显而易见的，那么就不要使用它。

切勿使用断言来测试程序的外部条件，例如文件的存在性、网络的可用性或用户输入的正确性。断言会测试程序的内部状态，以确保其在规范范围内。当断言失败时，说明程序在某种意义上已经脱离了轨道，进入了设计时未考虑到的正常运行状态。因此，断言失败表明程序存在缺陷。外部故障并不是系统中的错误（它们甚至可能根本就不是错误！），你无法事先对程序做出任何修改来防止它们的发生。外部故障应使用异常来处理。

如前所述，许多断言机制都设计成仅在测试和调试期间执行断言，并在程序发布给用户时关闭断言。（重申一下，Node 断言的情况并非如此，它不能被禁用；但例如 Java 断言实际上默认是禁用的）。由于断言有时会被禁用，因此好的做法是确保程序的正确性不取决于断言表达式是否被执行。尤其是，断言表达式不应有副作用。例如，如果您想从列表中弹出一个元素，并断言在弹出之前列表是非空的，就不要这样写：

```ts
// don't do this:
assert(list.pop());
```
如果断言被禁用，整个表达式将被跳过，并且不会从列表中弹出任何项。可以这样写
```ts
const found = list.pop();
assert(found);
```

同样，如果条件语句或switch没有涵盖所有可能的情况，使用断言检查并阻止非法情况也是不错的做法。

```ts
switch (vowel) {
  case 'a':
  case 'e':
  case 'i':
  case 'o':
  case 'u': return 'A';
  default: assert.fail('should never get here');
}
```

### Incremental development

incremental development是将错误定位到程序极小部分的好方法。每次只编写一点程序，并在继续编写之前彻底测试这一点。这样，当你发现错误时，它更有可能出现在你刚刚编写的部分，而不是一大堆代码中的任何地方。

我们在测试课上介绍了两种有助于实现这一点的技术：

* 单元测试：当你孤立地测试一个模块时，你可以确信你发现的任何错误都在该单元中，或者可能在测试用例本身中。
* 回归测试：在大型系统中添加新功能时，尽可能频繁地运行回归测试套件。如果测试失败，错误很可能就在你刚刚修改的代码中。
我们还谈到了版本控制。经常添加、提交和推送不仅能让你的工作免于断电，也是保存工作中无错误代码的好方法。如果你经常添加、提交和推送工作代码，那么一旦出现错误，你就可以将你的修改与上次保存的工作版本进行比较，并根据差异进行调试。

### Modularity & encapsulation

你还可以通过更好的软件设计来定位错误。

Modularity是指将系统划分为多个组件或模块，每个组件或模块都可以独立于系统的其他部分进行设计、实施、测试、推理和重用。模块化系统的反面是单体系统--体积庞大，所有部件相互缠绕、相互依赖。

由一个个很长的函数组成的程序是单体的--更难理解，也更难隔离其中的错误。相比之下，分成小函数和小class的程序模块化程度更高。

封装是指在模块周围筑起围墙，使模块只对自己的内部行为负责，系统其他部分的错误不会破坏模块的完整性。

其中一种encapsulation是访问控制，即使用 public 和 private 来控制变量和方法的可见性和可访问性。任何代码都可以访问公共变量或方法（假设包含该变量或方法的类也是公共的）。私有变量或方法只能由同类中的代码访问。尽可能保持私有，尤其是变量的私有，可以提供封装性，因为它限制了可能无意中导致错误的代码。

另一种encapsulation来自变量的作用域。变量的作用域是程序中定义该变量的部分，即表达式和语句可以引用该变量。函数参数的作用域是函数的主体。局部变量的作用域从其声明到下一个结尾大括号。尽可能缩小变量的作用域，可以更容易地推理出程序中可能存在的错误。例如，假设有这样一个循环

```ts
for (i = 0; i < 100; ++i) {
    ...
    doSomeThings();
    ...
}
```
...然后你发现，这个循环一直在运行——但是i 从未达到 100。在某个地方，有人正在改变 i。如果 i 被声明为全局变量，就像下面这样：

```ts
let i: number;
...
for (i = 0; i < 100; ++i) {
    ...
    doSomeThings();
    ...
}
```
......那么它的作用域就是整个程序。它可能会在程序的任何地方发生变化：被 doSomeThings()、被 doSomeThings() 调用的其他方法、被运行完全不同代码的并发线程。但如果将 i 声明为作用域狭窄的局部变量，就像下面这样：

```ts
for (let i = 0; i < 100; ++i) {
    ...
    doSomeThings();
    ...
}
```
......唯一可以修改 i 的地方就是 for 语句中，事实上，只有在我们省略的......部分。你甚至不必考虑 doSomeThings()，因为 doSomeThings() 无法访问这个局部变量。

将变量的作用域最小化是实现 bug localization 的有效方法。以下是一些适用于 TypeScript 的规则：

* 始终在 for 循环初始化器中声明循环变量,不要在循环之前声明：
  ```ts
  let i: number;
  for (i = 0; i < 100; ++i) {
  ```
这使得变量的作用域是包含这段代码的整个外层大括号代码块的其余部分，你应该这样做：
```ts
for (let i = 0; i < 100; ++i) {
```
这使得 i 的作用域仅限于 for 循环。

* 一定要使用 const 或 let，千万不要使用 var。 const 或 let 变量的作用域是围绕它的最小的大括号块，而 var 声明的作用域是围绕它的整个函数（顺便提一下，Python 局部变量使用的也是同样的规则）。你会在网上看到很多使用 var 的旧 JavaScript 代码，请避免使用。现代 JavaScript/TypeScript 编程总是使用 const 或 let。

* 只有在第一次需要变量时，才在最内层的大括号块中声明变量。将变量声明放在包含所有需要使用该变量的表达式的最内层代码块中。不要在函数开始时声明所有变量--这会使变量的作用域变得不必要的大。

* 避免使用全局变量。使用全局变量是一个非常糟糕的主意，尤其是当程序变大时。使用全局变量作为在程序的多个部分之间共享某些数据的快捷方式可能很诱人，但千万不要上当。最好是直接将参数传递到需要它的代码中，而不是将它放在全局空间中，因为它可能在无意中被重新分配。




## Reproduce the bug
假设有人向你报告了你编写的软件中的一个错误（例如，在提交 pset 的 alpha 版本后，一个隐藏测试用例失败了）。这在软件工程生涯中是不可避免的。

首先要找到一个小型、可重复的测试用例来重现故障。如果错误是通过regression testing发现的，那你就走运了；你的测试套件中已经有了一个失败的测试用例。如果错误是由用户报告的，则可能需要花费一些精力来重现错误。对于图形用户界面和多线程程序来说，如果一个错误依赖于事件或线程执行的时间，那么它就很难持续重现。

尽管如此，您为使测试用例小巧且可重复而付出的任何努力都将得到回报，因为在您查找错误并开发修复方法时，您必须反复运行测试用例。此外，在成功修复漏洞后，您还需要在回归测试套件中保留该测试用例，这样漏洞就不会再出现了。一旦有了针对错误的测试用例，让这个测试正常工作就成了你的目标。

下面是一个例子。假设您编写了这样一个函数

```ts
/**
 * Find the most common word in a string.
 * @param text string containing zero or more words, where a word
 *     is a string of alphanumeric characters bounded by nonalphanumerics.
 * @returns a word that occurs maximally often in text, ignoring alphabetic case.
 */
function mostCommonWord(text: string): string {
    ...
}
```

用户将莎士比亚戏剧的全部文本传入你的方法（类似 mostCommonWord(allShakespearesPlaysConcatenated)），结果发现该方法返回的不是 "the "或 "a "等可预测的常见英文单词，而是一些意想不到的单词，可能是 "e"。

莎士比亚的戏剧有 100,000 行，包含 800,000 多个单词，因此这种输入调试起来会非常痛苦。如果首先将错误输入的大小减小到可以处理的程度，但仍能显示相同（或非常相似）的错误，调试就会变得更容易：

* 莎士比亚的前半部分是否显示了相同的错误？（然后继续减半）！二进制搜索通常是一种很好的技术。下文将详细介绍）。
* 一个剧本是否有相同的错误？
* 一个独白是否有相同的错误？
找到一个小测试用例后，用这个小测试用例找到并修复错误，然后回到原来的错误输入，确认你修复了同样的错误。